### Benefits of using Ollama

No charges
Data stays local

### Disadvantages 
Not Powerful as OpenAI



In [45]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [36]:
# Model Constant
MODEL = "llama3.2"

In [46]:
import ollama

# Try pulling the model
try:
    ollama.pull("llama3.2")
except Exception as e:
    print(f"Error pulling model: {e}")

In [47]:
class Website:
    def __init__(self,url):
        self.url = url
        self.title = None
        self.text = None
        
        #set custom headers, especially user-agent, to avoid issues with some websites blocking the request.
        
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        try:
            # Make an HTTP request to fetch page content
            response = requests.get(url,headers)
            response.raise_for_status() # raise an exception for HTTP errors
            
            # Parst the website content using BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Extract the title
            self.title = soup.title.string if soup.title else 'No Title Found'

            # Remove irrelevant tags
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            
            # Extract text, stripping unnecessary whitespace and adding newlines between elements
            self.text = soup.body.get_text(separator="\n", strip=True)

        except requests.exceptions.RequestException as ex:
            print(f"Error fetching {url}: {ex}")
        except Exception as ex:
            print(f"Error processing the website content: {ex}")

    # Returns the title of the webpage
    def get_title(self):
        return self.title
    
    # Returns the cleaned text content of the webpage
    def get_text(self):
        return self.text
    
    def __str__(self):
        return f"{self.title} ({self.url})"
        

In [38]:
# Let's try one out

mywebsite = Website("https://cnn.com")
print(mywebsite.title)
print(mywebsite.text[:200])

Breaking News, Latest News and Videos | CNN
CNN values your feedback
1. How relevant is this ad to you?
2. Did you encounter any technical issues?
Video player was slow to load content
Video content never loaded
Ad froze or did not finish loadi


In [48]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [49]:
def user_prompt_for(mywebsite):
    user_prompt = f"You are looking at a website titled {mywebsite.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += f"URL: {mywebsite.url}\n"
    return user_prompt

In [50]:

def messages_for(mywebsite):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(mywebsite)}
    ]

In [51]:
#  call the Ollama function instead of OpenAI

def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [52]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [53]:
display_summary("https://cnn.com")

### Website Summary: CNN

#### Overview
-----------------

*   **Title:** Breaking News, Latest News and Videos | CNN
*   **Type:** News Network
*   **Category:** Online Media

#### Featured Content
--------------------

*   **Latest Headlines:**
    *   COVID-19 pandemic updates
    *   Ongoing global conflicts
    *   Breaking news on various topics
*   **Top Stories:**
    *   US Politics and Government
    *   Business News
    *   Sports News

#### Notable Features
---------------------

*   **News Videos:** CNN's original video content, covering various news topics.
*   **Interactive Content:** Engaging features, such as polls and quizzes, to enhance user experience.

Note: The provided summary is based on the main content of the website and excludes navigation-related sections.

In [54]:
display_summary("https://yahoo.com")

**Summary of Yahoo Website**
=====================================

### Overview

Yahoo is a comprehensive online platform providing access to various services including email, weather forecasting, search engine, news, finance, sports, and video content.

### Key Features

* **Email**: Personal email accounts with storage space and spam filtering
* **Weather**: Current weather conditions, forecasts, and radar imagery for worldwide locations
* **Search**: A search engine allowing users to find information on the web
* **News**: Breaking news headlines from various sources
* **Finance**: Real-time stock market data, financial news, and investment tools
* **Sports**: Live scores, news, and updates from different sports leagues and teams
* **Videos**: Video content from multiple sources, including news and entertainment

### News and Announcements

Unfortunately, the provided URL does not contain any recent news or announcements. However, Yahoo's news section (https://news.yahoo.com) provides up-to-date information on various topics.

### Notes

The website seems to have undergone some redesigns in recent years, with a more modern and minimalistic layout.

In [55]:
display_summary("https://cnbc.com")

**Summary of CNBC Website**
====================================

* **Tagline:** "Worldwide leader in business news and information"
* **Content:**
	+ **Main Sections:**
		- Stock Markets
		- Business News
		- Financials
		- Earnings
	+ **News and Announcements:**
		- Breaking news on market trends, economic data, company earnings, and business developments.
		- In-depth coverage of the stock market, including stocks, ETFs, mutual funds, and commodities.
		- Analysis and commentary from top business journalists and experts.
	+ **Features and Tools:**
		- Real-time market data and charts
		- Stock screeners and portfolio management tools
		- Personal finance advice and planning resources

**Notable News and Announcements:**

* The US stock market is experiencing a correction due to rising interest rates and inflation concerns.
* A major tech company announced strong quarterly earnings, beating analyst expectations.
* The global economy is expected to slow down in 2024 due to geopolitical tensions and trade disputes.

Please note that the news section on CNBC is constantly updating, so this summary might not be comprehensive or up-to-date.